# Vickrey Auction

> `Mercury` is an actor-based modelling framework for games on graphs.

This notebook will go over an example implementation of a Vickrey Auction in `mercury`.

The objective is to cover the key concepts in implementing your own games.

In [ ]:
import numpy as np
import mercury as me
import matplotlib.pyplot as plt

from dataclasses import dataclass

plt.style.use('bmh')

## Message Types:

In [ ]:
@dataclass(frozen=True)
class PlaceBid(me.Message):
    price: float

## Actor Types:

In [ ]:
class Bidder(me.actors.SimpleSyncActor):
    """Custom bidding actor.

    This class does nothing, but acts as a wrapper around the SimpleSyncActor
    for the sake of naming.
    """
    pass

In [ ]:
class Auctioneer(me.actors.Actor):
    """Vickrey auction mechanism.
    
    This class waits to receive all messages in a single timestep, sorts them,
    and then responds to the winner with a FlowMessage. If multiple messages are
    sent from a single Bidder, then the lowest price is taken.

    The winner is defined as the best price, but the execution is applied at
    the second best price.
    """
    def handle_requests(self, ctx: me.Network.Context) -> None:
        def get_bid(nid):
            reqs = ctx.requests_from(nid)

            return min(reqs, key=lambda req: req.price)

        bids = [(nid, get_bid(nid)) for nid in ctx.neighbour_ids]
        bids.sort(key=lambda x: x[1].price)

        flow = me.Flow(inv=1.0, cash=-bids[-2][1].price)

        ctx.send_response(bids[-1][0], me.FlowMessage(self.id, flow))

## Custom Network Type:

In [ ]:
class VickreyAuction(me.Network):
    """Wrapper around Network to simplify Vickrey auction simulations.

    This class does two things:
        1. Simplify the creation and connection of the Actioneer and Bidder actors.
        2. Add a helper for placing a set of bids messages simultaneously.
    """
    def __init__(self, n_bidders: int) -> None:
        me.Network.__init__(self, actors={
            'auctioneer': Auctioneer('auctioneer'),
        })

        for i in range(n_bidders):
            bidder_id = 'bidder_{}'.format(i)

            self.add_actor(Bidder(bidder_id))
            self.add_connection(bidder_id, 'auctioneer')

    def send_bids(self, bid_map):
        for bidder_id, bid in bid_map.items():
            req = PlaceBid(bidder_id, bid)

            self.context_for(bidder_id).send_request('auctioneer', req)

        return self

## Run Simulation:

In [ ]:
N_ROUNDS = 10000
N_BIDDERS = 10

In [ ]:
game = VickreyAuction(N_BIDDERS)

for _ in range(N_ROUNDS):
    game.send_bids({
        'bidder_{}'.format(i): max(np.random.normal(loc=i, scale=2.0), 0.0)
        for i in range(N_BIDDERS)
    }).resolve()

for i in range(N_BIDDERS):
    bidder = game['bidder_{}'.format(i)]
    avg_price = bidder.cash.current / bidder.inventory.current if abs(bidder.inventory) > 0.0 else 0.0
    market_share = bidder.inventory.current / N_ROUNDS

    print(
        '{}:'.format(bidder.id),
        avg_price,
        '\t({:.0%})'.format(market_share)
    )